In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport model

In [4]:
import skorch

In [5]:
import numpy as np

In [6]:
import keras

Using TensorFlow backend.


In [7]:
import torch

In [8]:
torch.cuda.is_available()

True

In [9]:
dataset = "fashion-mnist"

In [10]:
dataset_config = {
    'size': 32,
    'channels': 1,
    'classes': 10
}

In [11]:
def error_rate(net, x=None, y=None, test_batch_size = 16):
    number_of_test_images = x_test.shape[0]
    number_of_error_images = 0
    for start in range(0, number_of_test_images, test_batch_size):
        end = min(start + test_batch_size, number_of_test_images)
        number_of_error_images += int((net.predict(x_test[start:end]) != y_test[start:end]).sum())
    return number_of_error_images / number_of_test_images

net = skorch.NeuralNetClassifier(
    module=model.Offnet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=24,
        widen_factor=10,
    ),
    criterion=torch.nn.CrossEntropyLoss,
    batch_size=16,
    max_epochs=10,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('progress_bar', skorch.callbacks.ProgressBar()),
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/offnet.pt", monitor=None))
    ],
    device='cuda' if torch.cuda.is_available() else 'cpu',
    #train_split=None, # Workaround for https://github.com/dnouri/skorch/issues/218
    lr=1e-04,
)
net.initialize()

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Offnet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): OffBlockGroup(
        (sequential): Sequential(
          (0): OffBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(16, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(144, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): FakeConv2d(
              (deform): Deform2d(
                (conv): Conv2d(160, 18, kernel_size=(1, 1), stride=(1, 1))
              )
              (conv): Conv2d(1440

In [12]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [13]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [14]:
def error_rate(test_batch_size = 16):
    number_of_test_images = x_test.shape[0]
    number_of_error_images = 0
    for start in range(0, number_of_test_images, test_batch_size):
        end = min(start + test_batch_size, number_of_test_images)
        number_of_error_images += int((net.predict(x_test[start:end]) != y_test[start:end]).sum())
    return number_of_error_images / number_of_test_images

In [ ]:
for _ in range(200):
    net.fit_loop(x_train, y_train)
    prediction_test = net.predict(x_test)
    error = (prediction_test != y_test).mean()
    print("error: ", error_rate())


Checkpoint! Saving model to models/untrainable_wrn1.pt.
  epoch    train_loss    valid_acc    valid_loss        dur
-------  ------------  -----------  ------------  ---------
      1        0.4681       0.8920        0.3074  1343.9497
